<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TestOUX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemple d'apprentissage du OUX (XOR) avec TensorFlow

Apprendre le ET pour réseau de neurone demande l'intervention d'une entrée supplémentaire appelée le BIAS
([voir le ET par réseaux de neurones](https://github.com/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TestET.ipynb))

Le tableau du OU Exclusif sans et avec Bias est :

|a|b|a oux b|le tableau avec bias -> |bias|a|b|a oux b|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|0|0|0| |1|0|0|0|
|0|1|1| |1|0|1|1|
|1|0|1| |1|1|0|1|
|1|1|0| |1|1|1|0|


*Théoriquement, en 1 couche, l'apprentissage du OUX par réseau de neurones est alors impossible.*

En effet, la couche n'est consituée que de 1 neurone (1 sortie), ses entrées sont les valeurs `bias`, `a` et `b`.
`wbias`, `wa` et `wb` étant les poids affectés à ces valeurs, il faut vérifier : 
 - `f(bias)` tend vers 0
 - `f(bias + wb)` tend vers 1
 - `f(bias + wa)` tend vers 1
 - `f(bias + wa + wb)` tend vers 0  --> **contradiction !**
 

*Vérifions le...*



---
**Importer les librairires**

In [1]:
#keras : Python Deep Learning library
import tensorflow.keras as keras
#prevision d'utiliser un réseau en couches séquentielles
from tensorflow.keras.models import Sequential
#prevision d'utiliser des couches totalement connectées la précédente
from tensorflow.keras.layers import Dense
#utilisation de la classique librairie pour tableaux, ...
import numpy as np

### Définir les entrées et sorties attendues
- a OUX b est vraie seulement si un seule des deux variable a ou b est vraie


In [2]:
# a et b sont les seules entrées
entrees = np.array([[0,0],[0,1],[1,0],[1,1]], float)

# une seule sortie
sorties = np.array([[0],[1],[1],[0]], float)

---
## 1. test du OUX - SANS COUCHE CACHEE


### 1.1. Choisir le modèle de réseau
- ici les couches sont séquentielles

In [3]:
model = Sequential()

### 1.2. Définir l'architecture du réseau
- ici une seule couche constituée de 1 neurone en sortie, 
- de 3 neurones en entrée (1 pour chaque valeur, +1 pour le Bias), 
- utilisation de la sigmoïde comme fonction d'activation

In [4]:
model.add(Dense(1, input_dim=2, use_bias=True, activation='sigmoid'))

### 1.3. Compiler le  réseau
Ici, on précise que 
  - l'algo de correction d'erreur est 'Adamax', 
  - l'erreur calculée est la moyenne des valeurs absolues des erreurs commises

In [5]:
model.compile(optimizer='adamax', loss='MSE')

### 1.4. Entraîner le réseau 
- et on lance 10000 cycles d'apprentissage

In [6]:
model.fit(entrees, sorties, verbose=0, epochs=1000)

### 1.5. Vérifier le réseau
Etape facultative, en général ***on teste le réseau sur d'autres exemples***. 
- Ici, on n'en a pas. Alors on lui demande de calculer la sortie pour chaque exemple de l'ensemble d'entraînement

In [7]:
predictions = model.predict(entrees)

### 1.6. Affichage des résultats
Ici pas de nécessité de graphique d'évolution de l'erreur.
On affiche les entrées, la sortie attendue, la sortie calculée.. et l'erreur..

In [8]:
def verification(bias=False):
    print("verification")
    for i in range(0, len(entrees)):
        print(entrees[i][0], " - ", entrees[i][1], " attendu ", sorties[i], " trouvé ",  predictions[i])

verification()

loss = model.evaluate(entrees, sorties,verbose=0)
print("perte=",loss)

verification
0.0  -  0.0  attendu  [0.]  trouvé  [0.42475054]
0.0  -  1.0  attendu  [1.]  trouvé  [0.43814373]
1.0  -  0.0  attendu  [1.]  trouvé  [0.5446196]
1.0  -  1.0  attendu  [0.]  trouvé  [0.5581254]
perte= 0.25374269485473633


**Un beau "plantage"**, comme cela était attendu

Ajoutons maintenant une couche cachée..

---
## 2. test du OUX - AVEC COUCHE CACHEE


### 2.1. Choisir le modèle de réseau
- ici les couches sont séquentielles

In [9]:
model = Sequential()

### 2.2. Définir l'architecture du réseau*** 
- une première couche composée de 
  - 3 neurones en entrée : 2 neurones pour les 2 variables, plus le neurone BIAS
  - **et maintenant 4 neurones en sortie !!**
- une **couche intermédiaire**
  - *implicitement de 4 neurones en entrée* (ceux de la couche précédente) et 
  - de 1 neurone en sortie (a OUX b)


In [10]:
#une premiere couche constituée de 2 neurones en sortie, de 3 neurones en entrée (2 pour les valeurs + 1 Bias), fonction d'activation = tangente hyperbolique
model.add(Dense(4, input_dim=2, use_bias=True, activation='tanh'))

#une seconde couche constituée de 1 neurone en sortie (et implicitement de 2 neurones en entrée), fonction d'activation = tangente hyperbolique
model.add(Dense(1, activation='tanh'))


### 2.3. Compiler le  réseau
- ici, on précise que l'algo de correction d'erreur est *'adam'*, et que l'erreur calculée est la moyenne des valeurs absolues des erreurs commises

In [11]:
model.compile(optimizer='adam', loss='MSE')

### 2.4. Entraîner le réseau 
- ici on ne le fait pas 'parler' (verbose=0), et on "ne lance plus que" 2000 cycles d'apprentissage

In [12]:
model.fit(entrees, sorties, verbose=0, epochs=2000)

### 2.5. Vérifier le réseau 
- Etape nécessaire; en général on le teste sur d'autres exemples. Ici, on lui demande de calculer la sortie pour chaque exemple de l'ensemble d'entraînement

In [13]:
predictions = model.predict(entrees)

### 2.7. Affichage des résultats
- pas de courbe d'erreurs ici, on se contente d'afficher les entrées, la sortie attendue et la sortie calculées; et on affiche l'erreur

In [14]:
verification()

loss = model.evaluate(entrees, sorties,verbose=0)
print("perte=",loss)

verification
0.0  -  0.0  attendu  [0.]  trouvé  [0.01189649]
0.0  -  1.0  attendu  [1.]  trouvé  [0.91050607]
1.0  -  0.0  attendu  [1.]  trouvé  [0.86376584]
1.0  -  1.0  attendu  [0.]  trouvé  [0.02558656]
perte= 0.006841277703642845


**Importance de la couche intermédiaire validée !!!**
---